In [1]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future

In [5]:
#!pip uninstall h2o -y

Uninstalling h2o-3.26.0.2:
  Successfully uninstalled h2o-3.26.0.2


In [6]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 123.6MB 666kB/s eta 0:00:013.6MB 666kB/s eta 0:00:01
  Created wheel for h2o: filename=h2o-3.26.0.3-py2.py3-none-any.whl size=123649963 sha256=db35ef45e884508929ecf1920f70451b9e8e2aa30ce8788814a1d536c5655d1b
  Stored in directory: /home/raffieey/.cache/pip/wheels/4a/a6/8d/0880c51d5aa9746a87f61d9b66d8a98e7b9cf400c81ffc944e
Successfully built h2o


In [7]:
import h2o

In [8]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.3" 2019-04-16; OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.10.1); OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.10.1, mixed mode, sharing)
  Starting server from /home/raffieey/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpwfhc4qzk
  JVM stdout: /tmp/tmpwfhc4qzk/h2o_raffieey_started_from_python.out
  JVM stderr: /tmp/tmpwfhc4qzk/h2o_raffieey_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Asia/Kuala_Lumpur
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_raffieey_3r5lr4
H2O cluster total nodes:,1
H2O cluster free memory:,1.896 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [49]:
from pmlb import classification_dataset_names, regression_dataset_names,fetch_data
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from pytictoc import TicToc

t = TicToc()
sel_clss_dtst = ['adult','agaricus-lepiota', 'churn', 'nursery', 'satimage','texture']
sel_rgrs_dtst = ['294_satellite_image','218_house_8L', '227_cpu_small', '503_wind', '344_mv','215_2dplanes']

for class_dtst in sel_clss_dtst:
    X, y = fetch_data(class_dtst, return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=69, stratify=y)
    X_train = h2o.H2OFrame(X_train)
    X_test = h2o.H2OFrame(X_test)
    y_train = h2o.H2OFrame(y_train).asfactor()
    y_test = h2o.H2OFrame(y_test).asfactor()
    aml = H2OAutoML(seed=69)
    t.tic()
    aml.train(x=X_train.columns, y=X_train.cbind(y_train)[:,-1].columns[0], training_frame=X_train.cbind(y_train))
    t.toc()
    lb = aml.leaderboard
    lb.head(rows=lb.nrows)
    preds[class_dtst] = aml.leader.predict(X_test)
 

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Elapsed time is 3408.772573 seconds.
stackedensemble prediction progress: |████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Elapsed time is 3279.674860 seconds.
gbm prediction progress: |█████████